## **This is the tutorial for using Groq**

- Groq is an open source platform where all the open source llm models are stored and we can use them using groq api key.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq

model = ChatGroq(model = "Gemma2-9b-It", groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023A0355FED0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023A03762990>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

- The system message is the prompt which is given to LLM to perform the task
- The human message is the actual text given to the LLM to perform the task on.

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from english to Marathi"),
    HumanMessage(content="Hello, how are you?")
]

response = model.invoke(messages)
response

AIMessage(content='नमस्ते, तुम्ही कसे आहात? (Namaste, tumhi kase aahate?) \n\n\n* **नमस्ते** (Namaste) - Hello\n* **तुम्ही** (Tumhi) - You\n* **कसे** (Kase) - How\n* **आहात?** (Aahate?) - Are you? \n\nThis is a common way to greet someone in Marathi. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 22, 'total_tokens': 118, 'completion_time': 0.174545455, 'prompt_time': 0.002149945, 'queue_time': 0.23139525400000002, 'total_time': 0.1766954}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ea1a6032-c3a2-4e6a-8c7d-3d96dfbf6f57-0', usage_metadata={'input_tokens': 22, 'output_tokens': 96, 'total_tokens': 118})

This StrOutputParser is used to just the get the context or answer from the above response, and it will ignore all the other values like metadata or token etc.

In [11]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

parser.invoke(response)

'नमस्ते, तुम्ही कसे आहात? (Namaste, tumhi kase aahate?) \n\n\n* **नमस्ते** (Namaste) - Hello\n* **तुम्ही** (Tumhi) - You\n* **कसे** (Kase) - How\n* **आहात?** (Aahate?) - Are you? \n\nThis is a common way to greet someone in Marathi. \n'

## **LCEL (LangChain Expression Language)**

This is used to chain components together as in above we can chain model and StrOutputParser, so basically we are chaining the events

In [13]:
chain = model | parser
chain.invoke(messages)

'नमस्ते, तुम्ही कसे आहात? (Namaste, tumhi kase ahat?) \n'

## **Prompt Template**

In [17]:
from langchain_core.prompts import ChatPromptTemplate


generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages(
    [("system", generic_template),
     ("user", "{text}")]
)

result = prompt.invoke({"language": "Hindi", "text": "Hello, you are so smart!"})
print(result)
result.to_messages()


messages=[SystemMessage(content='Translate the following into Hindi:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, you are so smart!', additional_kwargs={}, response_metadata={})]


[SystemMessage(content='Translate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, you are so smart!', additional_kwargs={}, response_metadata={})]

**Chaining these prompts model and parser toether**

In [19]:
chain = prompt|model|parser

chain.invoke({"language": "hindi", "text": "Hello"})

'नमस्ते (Namaste) \n'